In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn import cross_validation
from sklearn.svm import LinearSVC
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import pairwise_distances
import os

In [89]:
import sqlite3


In [117]:
import pandas as pd
import numpy as np
%matplotlib
import matplotlib.pyplot as plt

Using matplotlib backend: Qt4Agg


/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Tests de performance.

Cosas que probe: 
    - Norma l1: funciona mucho peor
    - Sacar acentos: funciona casi igual, un poco peor
    - unigramas vs bigramas no da nada muy diferente
 

In [108]:
# Cargar datos
data = pd.read_excel('Informe 93 - HCDN (Respuestas) (1) (1).xlsx',sheetname='Respuestas de formulario 1')
ES_stopwords = pd.read_csv('ES_stopwords.txt',header=None, encoding='utf-8')
ES_stopwords = list(np.squeeze(ES_stopwords.values))
# Sacar las Pretuntas o temas vacios
gi = ~data.Tema.isnull()
gi = gi & (~data.Pregunta.isnull())
preguntas = data.Pregunta[gi]
temas = data.Tema[gi]

# Funcion para calcular la precision pero considerando las primeras N opciones
def calculate_n_accuracy(X_test, y_test, svm, maxN):
    correctas_capturadas = []
    for N in range(1,maxN):
        correctas = []
        for test_row, tema in zip(X_test,y_test):
            relevancia = np.squeeze(svm.decision_function(test_row))
            prediccion = svm.predict(test_row)[0]
            I = np.argsort(relevancia)
            primerasN = svm.classes_[I[-N:]]
            if tema in primerasN:
                correctas.append(1)
            else:
                correctas.append(0)
        correctas_capturadas.append(np.mean(correctas))
    return correctas_capturadas

NameError: name 'pd' is not defined

### TF-IDF de palabras

In [87]:
textvec_palabras = TfidfVectorizer(input='content', ngram_range=(1,1),  min_df=1,
                                   norm='l2', stop_words=ES_stopwords)
tfidf_mat = textvec_palabras.fit_transform(preguntas)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
                tfidf_mat, temas, test_size=0.5, random_state=0)
svm = LinearSVC()
svm.fit(X_train,y_train)
print "Score correctas: {:f}".format(svm.score(X_test,y_test))
y1 = calculate_n_accuracy(X_test, y_test, svm, maxN=10)


Score correctas: 0.486405


In [82]:
textvec_bigramas = TfidfVectorizer(input='content', ngram_range=(1,3), min_df=2,
                                   norm='l2', stop_words=ES_stopwords)
tfidf_mat = textvec_bigramas.fit_transform(preguntas)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
                tfidf_mat, temas, test_size=0.5, random_state=0)
svm = LinearSVC()
svm.fit(X_train,y_train)
print "Score correctas: {:f}".format(svm.score(X_test,y_test))
y2 = calculate_n_accuracy(X_test, y_test, svm, maxN=10)


Score correctas: 0.468278


In [83]:
# Plot curvas de ambos
plt.plot(np.arange(1,len(y1)+1),y1)
plt.plot(np.arange(1,len(y2)+1),y2)
plt.xlabel('Primeras n opciones')
plt.ylabel('Fraccion correctas')

In [3]:
archivos_excel =[unicode(archivo, 'utf') for archivo in os.listdir('.') if 'xlsx' in archivo]

In [4]:
all_dataframes = []
for archivo in archivos_excel:
    all_dataframes.append(pd.read_excel(archivo, sheetname='Respuestas de formulario 1'))
all_questions = pd.concat([x.Pregunta for x in all_dataframes], keys=['94', '94FT','95', '93']).to_frame()


In [5]:
preg_cat = pd.read_excel('Informe 93 - HCDN (Respuestas) (1) (1).xlsx',sheetname='Respuestas de formulario 1')
preg_cat = preg_cat.sortlevel(0)
all_questions = all_questions.sortlevel(0)
all_questions.loc['93','tema'] = preg_cat.Tema.values
all_questions.dropna(subset=['Pregunta'], inplace=True)
ES_stopwords = pd.read_csv('ES_stopwords.txt',header=None, encoding='utf-8')
ES_stopwords = list(np.squeeze(ES_stopwords.values))
all_questions.dropna(subset=['Pregunta'], inplace=True) # Sacar preguntas vacias
longitud = all_questions.Pregunta.str.len()
all_questions = all_questions.loc[longitud.sort_values().index[50:]] # Sacar preguntas demasiado cortas

In [95]:
preg_cat.index = preg_cat.index.astype('int64')

In [117]:
all_questions.loc['93','tema'] = preg_cat.Tema.values

In [121]:
preg_cat.tail()

,Autor de la Pregunta,Numero,Órgano requerido,Tema,Subtema,2 o mas Ministerios?,Justificación o contexto,Pregunta
744,Interbloque Juntos Por Argentina,745.0,Ministerio de Turismo,EducaciónyDeportes,Programas educativos.,No,NaN,Qué medidas establecerá el Poder Ejecutivo par...
745,Interbloque Juntos Por Argentina,746.0,Ministerio de Turismo,Turismo,Política turística,No,NaN,Qué medidas adoptará el Poder Ejecutivo para m...
746,Interbloque Juntos Por Argentina,747.0,Ministerio de Justicia y Derechos Humanos,Turismo,Política turística,No,NaN,Qué medidas adoptará el Gobierno Nacional en l...
747,Interbloque Juntos Por Argentina,748.0,Concejo Nacional de Políticas Sociales,DesarrolloSocial,Género,No,NaN,Qué medidas adoptará el Gobierno Nacional para...
748,NaN,NaN,NaN,DesarrolloSocial,Género,NaN,NaN,NaN


In [8]:
textvec = TfidfVectorizer(input='content', strip_accents='ascii', norm='l2', stop_words=ES_stopwords, )
preguntas = all_questions.Pregunta
temas = all_questions.tema
matriz = textvec.fit_transform(preguntas)
#X_train, X_test, y_train, y_test = cross_validation.train_test_split(
#            matriz_io_a, temas, test_size=0.5, random_state=0)

In [ ]:
distancias = 

In [157]:
svm = LinearSVC()
i_tiene_tema = temas.notnull()
svm.fit(matriz[i_tiene_tema.values.nonzero()], temas[i_tiene_tema].values)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [193]:

i = np.random.randint(0,len(all_questions))
all_questions.iloc[i].Pregunta

u'\xbfEst\xe1 previsto alg\xfan tipo de incentivo tributario espec\xedfico a la actividad productiva de la regi\xf3n para tratar de compensar los mayores costos de transporte que se mantendr\xe1n hasta tanto la nueva  infraestructura planificada se concrete?'

In [174]:
# Sugerir primeras N 
relevancia = svm.decision_function(matriz[10])
I = np.argsort(relevancia)
N = 5
primerasN = svm.classes_[I[0,-N:]]

array([u'ObraP\xfablica', u'Justicia ', u'Pol\xedticaEcon\xf3mica',
       u'Empleo ', u'Energ\xeda '], dtype=object)

In [361]:
svmGDC= SGDClassifier(loss='hinge', penalty='l2',
                     alpha=1e-3, n_iter=5, random_state=42)



In [362]:
svm.fit(X_train,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [242]:


y = np.squeeze(svm.decision_function(X_test[0,:]))
I = np.argsort(y)
x = range(len(y))
plt.bar(x,y)

<Container object of 18 artists>

In [363]:
svm.score(X_test,y_test)

0.27190332326283989

In [234]:
svm.fit(X_train, y_train)
predicted = svm.predict(X_test)

In [206]:
text_clf = Pipeline([('tfidf', TfidfVectorizer(input='content', strip_accents='ascii', norm='l2')),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                     alpha=1e-3, n_iter=5, random_state=42)),
                    ])

In [280]:
correctas_capturadas = []
for N in range(1,7):
    correctas = []
    for test_row, tema in zip(X_test,y_test):
        relevancia = np.squeeze(svm.decision_function(test_row))
        prediccion = svm.predict(test_row)[0]
        I = np.argsort(relevancia)
        primerasN = svm.classes_[I[-N:]]
        if tema in primerasN:
            correctas.append(1)
        else:
            correctas.append(0)
    correctas_capturadas.append(np.mean(correctas))

In [281]:
plt.plot(correctas_capturadas)

In [237]:
svm.classes_

array([u'Agroindustria ', u'CienciayTecnolog\xeda ', u'Comunicaciones ',
       u'Defensa ', u'DesarrolloSocial ', u'Educaci\xf3nyDeportes',
       u'Empleo ', u'Energ\xeda ', u'Justicia ', u'Modernizaci\xf3n ',
       u'ObraP\xfablica', u'Pol\xedticaEcon\xf3mica', u'Producci\xf3n ',
       u'RelacionesExteriores ', u'SEDRONAR ', u'Salud  ', u'Seguridad ',
       u'Transporte '], dtype=object)

In [208]:
t = matriz[0,:].toarray()
I = np.squeeze(np.argsort(t,))

In [209]:

tokens = np.array(textvec.get_feature_names())
tokens[I[-20:]]

array([u'enargas', u'esfuerzo', u'consolidacion', u'universo', u'merituar',
       u'determinadas', u'invertido', u'recaudado', u'cuyana', u'cuanto',
       u'san', u'fondo', u'luis', u'usuarios', u'ni', u'obras',
       u'provincia', u'de', u'focegas', u'distribuidora'], 
      dtype='<U20')

In [164]:
print(preguntas[2])

a)	Importe total transferido desde dicha fecha hasta la actualidad por aplicación de aquellas, especificando el incremento producido en las transferencias diarias por cumplimiento de las resoluciones de la Corte Suprema de Justicia de la Nación.
b)	Estado de cumplimiento de las sentencias, en cuanto condenaron al Estado Nacional a pagar a la Provincia de San Luis las cantidades detraídas -con anterioridad al dictado de las mismas- en virtud de las normas impugnadas, con más sus intereses informando, en su caso, si se han determinado los montos y modalidades de pago e informando sobre tal punto.


In [282]:
len(preg_cat.Subtema.unique())

75

In [283]:
600./75

8.0

In [306]:
suma = np.squeeze(matriz.power(2).sum(axis=1)).T

In [9]:
D = pairwise_distances(matriz)


In [10]:
plt.matshow(D)

In [316]:
import scipy.cluster.hierarchy as sch



In [317]:
Y = sch.linkage(D, method='centroid')
Z = sch.dendrogram(Y, orientation='right')
index = Z['leaves']
D = D[index,:]
D = D[:,index]

In [338]:
D[D==0] = 100
i,j = np.unravel_index(np.argmin(D),D.shape)

In [19]:
cercanas = np.argsort(D[14])[:4]

In [21]:
all_questions.iloc[14]

Pregunta    ¿Se esperan más aumentos en los servicios públ...
tema                                                      NaN
Name: (94, 14), dtype: object

In [23]:
all_questions.iloc[cercanas].Pregunta

94    14     ¿Se esperan más aumentos en los servicios públ...
94FT  325    Con respecto a las tarifas, el Ministro Arangu...
95    431    ¿Podría detallarlas por concepto y jurisdicció...
      297    Que se informe a valor CIF de las importacione...
Name: Pregunta, dtype: object

In [334]:
print(preguntas[14])

¿Cuáles son las razones por las cuales la Ley Nº 26.586, sancionada en el año 2009, por la que se crea el PROGRAMA NACIONAL DE EDUCACION Y PREVENCION DE LAS ADICCIONES Y EL CONSUMO INDEBIDO DE DROGAS, no ha sido reglamentada? ¿En qué plazo se reglamentará la misma?


In [335]:
print(preguntas[15])

¿Qué campañas publicitarias tendientes a la prevención del consumo de drogas sintéticas y promoción de la salud de acuerdo a lo establecido en el Artículo 3º de la Ley Nº 23.358, tiene previsto realizar el Poder Ejecutivo Nacional? ¿En qué plazos?


In [364]:
for n in range(10):
    i,j = np.unravel_index(np.argmin(D),D.shape)
    D[i,j] = 100
    D[j,i] = 100
    print(preguntas.iloc[i])
    print('---------------------')
    print(preguntas.iloc[j])
    print('=====================')

- ¿Cuáles son los “cientos de fármacos obsoletos o sin efectividad clínica comprobada dentro de los listados” que informó el PAMI en su comunicado. Cuál o cuáles las enfermedades o dolencias para las cuales son recetados los medicamentos que se excluyeron el 100% de la cobertura.
---------------------
- ¿Qué implica que los “cientos de fármacos obsoletos o sin efectividad clínica comprobada dentro de los listados” que informó el PAMI en su comunicado  sean obsoletos o sin efectividad? ¿Quién determina dicha obsolescencia o falta de efectividad clínica?
Informe la cantidad de empleados del sector privado que aportan a los servicios de Seguridad Social de ANSES que residen en las provincias de Jujuy, Salta, Tucumán, Santiago del Estero y Catamarca, disgregado por departamento, sexo, edad, antigüedad y tamaño del establecimiento.-
---------------------
Informe la cantidad de Jóvenes beneficiarios del Programa Jóvenes con Más y Mejor Trabajo  que residen en las provincias de Jujuy, Salta, 

In [341]:
preguntas.iloc[1]

u'a)\tImporte total transferido desde dicha fecha hasta la actualidad por aplicaci\xf3n de aquellas, especificando el incremento producido en las transferencias diarias por cumplimiento de las resoluciones de la Corte Suprema de Justicia de la Naci\xf3n.\nb)\tEstado de cumplimiento de las sentencias, en cuanto condenaron al Estado Nacional a pagar a la Provincia de San Luis las cantidades detra\xeddas -con anterioridad al dictado de las mismas- en virtud de las normas impugnadas, con m\xe1s sus intereses informando, en su caso, si se han determinado los montos y modalidades de pago e informando sobre tal punto.'

In [344]:
import indicoio
indicoio.config.api_key = 'cab4e98f91beb5173d3e6bd53319ede3'

In [351]:
matriz_io = indicoio.text_features(list(preguntas.values))

/usr/local/lib/python2.7/dist-packages/urllib3/util/ssl_.py:120: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning


In [357]:
matriz_io_a = np.array(matriz_io)

(662, 300)

In [365]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
            preguntas, temas, test_size=0.5, random_state=0)

In [395]:
entrenamiento = pd.concat((X_train.apply(lambda x: x[:980]), y_train), axis=1)
testeo = pd.concat((X_test.apply(lambda x: x[:980]), y_test), axis=1)

In [396]:
entrenamiento.to_csv('entrenamiento.csv', encoding='utf8',index=False, header=False)
testeo.to_csv('testeo.csv', encoding='utf8',index=False, header=False)

In [392]:
entrenamiento.Pregunta.apply(lambda x: x[:980])

345    ¿Cuál es el costo actual del plan Costo del pl...
78     Sobre Instituto Nacional de Servicios Sociales...
319    Informe sobre una estimación de las pérdidas e...
5      Inflación: ¿Cuáles son las medidas monetarias,...
607    Según  índices de público conocimiento, las ve...
580    En la anterior visita del Jefe de Gabinete sol...
404    La Resolución 3878/16 de la AFIP establece un ...
482    ¿Cuál es la situación actual de las negociacio...
6      Indique la cantidad de empleados públicos naci...
171    Informe el detalle discriminado de las transfe...
664    - Por  qué motivo el Ministerio de Seguridad d...
165    Informe especialmente sobre los despidos en la...
606    Recientemente  el Poder Ejecutivo aplicó un 40...
671    - Con  respecto al informe “El estado del Esta...
23     Señor Jefe de Gabinete: en sus respuestas escr...
681    -  La última reforma al Impuesto a las gananci...
313    Ante el brote de Gripe A faltan insumos, no ha...
321    -¿Por qué (el Ministro d

# Testeos en un corpus estandar

Bajado de https://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups


In [2]:
tv = TfidfVectorizer(input='content', encoding='latin1', decode_error='strict', strip_accents='ascii', 
                     lowercase=True, preprocessor=None, tokenizer=None, analyzer='word', stop_words=None, 
                     ngram_range=(1, 1), max_df=1.0, min_df=3, max_features=None, 
                     vocabulary=None, binary=False, norm='l2', use_idf=True, 
                     smooth_idf=True, sublinear_tf=False)

In [3]:
import codecs
cats = os.listdir('./test_data/20_newsgroups/')
cats_totales = []
contenidos = []
for cat in cats:
    temp_list = os.listdir(os.path.join('./test_data/20_newsgroups/', cat))
    cats_totales = cats_totales + [cat]*len(temp_list)
    temp_list = map(lambda x: os.path.join('./test_data/20_newsgroups/',cat,x), temp_list)
    for filename in temp_list:
        with codecs.open(filename, encoding='latin1', mode='r') as content_file:
            lines = content_file.readlines()
            content = ''.join(lines[11:])
        contenidos.append(content)


In [4]:
tfidf_mat = tv.fit_transform(contenidos)

In [14]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
                svd_mat, cats_totales, test_size=0.5, random_state=0)

In [15]:
svm = LinearSVC()
svm.fit(X_train,y_train)
print "Score correctas: {:f}".format(svm.score(X_test,y_test))

Score correctas: 0.729973


In [8]:
nn = NearestNeighbors(algorithm='kd_tree')

In [123]:
nn.kneighbors(X_test, )

'auto'

In [10]:
from sklearn.decomposition import TruncatedSVD

In [11]:
tt = TruncatedSVD(n_components=100)

In [12]:
svd_mat = tt.fit_transform(tfidf_mat)

array([[ 0.13710171, -0.05409748, -0.08087499, ..., -0.02903356,
         0.02067871,  0.02060423],
       [ 0.18322294, -0.02305735, -0.03651633, ...,  0.01567197,
         0.03723891, -0.00134786],
       [ 0.20044775, -0.01260446, -0.04811918, ...,  0.01005423,
        -0.01316359,  0.01250547],
       ..., 
       [ 0.2322936 ,  0.03845939,  0.00920797, ..., -0.00431266,
         0.05547958,  0.0322802 ],
       [ 0.2529581 ,  0.11313888, -0.05474577, ..., -0.03139609,
         0.00155107,  0.00869457],
       [ 0.33897048,  0.17291207, -0.07150881, ..., -0.01301681,
         0.02055989,  0.00967401]])